# Analyze a Youtube video by asking the LLM
By [Lior Gazit](https://www.linkedin.com/in/liorgazit/)  

<a target="_blank" href="https://colab.research.google.com/github/LiorGazit/LLM_search_inside_youtube_videos/blob/main/Analyze_a_Youtube_video_by_asking_the_LLM.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Description of the notebook:**  
Pick a Youtube video that you'd like to understand what value it brings you without having to spend the time to watch all of it.  
For instance: an hour long lecture about a topic you are looking to learn about, and your goal is know whether it touches on all key points before dedicating time to watch it.  
This is with the intuition that if it were a PDF instead of a video, you'd be able to search through it.  

**Requirements:**  
* Open this notebook in a free [Google Colab instance](https://colab.research.google.com/).  
* This code picks OpenAI's API as a choice of LLM, so a paid **API key** is necessary.   

Install:

In [ ]:
!pip -q install --upgrade embedchain
!pip -q install pytube
!pip -q install openai==0.28.1

Imports:

In [2]:
import os
import textwrap
import pandas as pd
import json

from embedchain import App
from embedchain.config import ChromaDbConfig


#### Insert API Key

In [ ]:
my_api_key =

#### Save API Key to Environement Variable

In [5]:
os.environ["OPENAI_API_KEY"] = my_api_key

#### Pick the Youtube Video and Insert its URL

In [6]:
video_url = "https://www.youtube.com/watch?v=ySEx_Bqxvvo&ab_channel=AlexanderAmini"

### Set Up the Retrieval Mechanism:

In [7]:
lecture_RAG = App(db_config=ChromaDbConfig(allow_reset=True))
lecture_RAG.reset()
lecture_RAG.add(data_type="youtube_video", source=video_url)



Inserting batches from 0 to 27 in vector database.
Successfully saved https://www.youtube.com/watch?v=ySEx_Bqxvvo&ab_channel=AlexanderAmini (DataType.YOUTUBE_VIDEO). New chunks count: 27


'36efd615c73bd3a6d2ba7d19d6754078'

### Some Questions About the Content of the Video

In [8]:
print(lecture_RAG.query("Do they mention transformers? In what way? Tell me in 2-3 sentences."))

Yes, transformers are mentioned in the context. The speaker mentions that attention is the foundational mechanism of the Transformer architecture, which is a powerful concept in modern deep learning and AI. They also mention that they will be discussing self-attention and how it can be used as part of a larger neural network like a Transformer.


In [9]:
print(lecture_RAG.query("Do they mention attention?"))

Yes, the context mentions attention multiple times.


In [10]:
backprop_answer_english = lecture_RAG.query("Do they mention back propogation? Please provide 2-3 sentences that tell about it.")
print(backprop_answer_english)

Yes, the passage mentions back propagation multiple times. It explains that in the case of recurrent neural networks (RNNs), back propagation is done through each individual time step in the sequence, propagating the loss from the current time step back to the beginning of the sequence. It also mentions that back propagation through time is a tricky algorithm to implement due to the repeated computations and multiplications of weight matrices, which can lead to the problem of exploding gradients.


#### Translate the Last Response to Hindi

In [11]:
print(lecture_RAG.query(f"Please translate this answer from English to Hindi: <{backprop_answer_english}>. Make sure to translate properly with the appropriate technical terms."))

हाँ, पाठ में कई बार बैक प्रोपेगेशन का उल्लेख है। यह स्पष्ट करता है कि पुनरावृत्ति न्यूरल नेटवर्क (आरएनएन) के मामले में, बैक प्रोपेगेशन को प्रत्येक व्यक्तिगत समय स्टेप में किया जाता है, जिससे लॉस को वर्तमान समय स्टेप से शुरुआत तक प्रसारित किया जाता है। इसके अलावा, यह उल्लेख करता है कि समय के माध्यम से बैक प्रोपेगेशन एक कठिन एल्गोरिदम है जिसे कई बार की गणना और वजन मैट्रिक्स के गुणन के कारण किया जाता है, जो विस्फोटित ग्रेडिएंट्स की समस्या का कारण बन सकता है।


#### Translate the Last Response to Tamil

In [12]:
print(lecture_RAG.query(f"Please translate this answer from English to Tamil: <{backprop_answer_english}>. Make sure to translate properly with the appropriate technical terms."))

ஆமாம், இந்த பதிவில் பின்வரும் பல முறைகளில் மீள்பதிவு குறித்து குறிப்பிடுகின்றது. மீள்பதிவு நிரல் வலிமை வலிமையுடன் இணைக்கப்பட்ட நேர படி (RNNs) உள்ளது, மீள்பதிவு ஒவ்வொரு துணை நேர படியிலும் நடக்கும், தற்போதைய நேர படியிலிருந்து தொடக்கத்திற்கு மீள்பதிவு செய்யும். இதுவரை மீள்பதிவு மூலம் பிழைகளை பின்வரும் துணை நேர படியில் பின்வரும் முழுமையான பிழையை உருவாக்குகின்றது. இதுவரை மீள்பதிவு மூலம் பிழைகளை பின்வரும் துணை நேர படியிலிருந்து தொடக்கத்திற்கு மீள்பதிவு செய்யும் என்பது மிகுந்த பிரச்சினையான அல்காரிதமாகும். மீள்பதிவு மூலம் பிழைகள் பரவும்போது நமது நிரல்களை மேம்படுத்த மற்றும் பயன்படுத்த வேண்டிய பிரதியை கணக்கிட நாங்கள் மிகுந்த எண்ணங்களை பல முறைகளில் மீண்டும் மீண்டும் பயன்படுத்த வேண்டும். இது எப்படி பிரச்சினையாக இருக்கலாம் என்று எங்கள் மிதமாக்குகின்றது என்பதை


### The Video's Text that the LLM Can Use to Answer:

In [13]:
lecture_RAG.db.get()['documents']

["Hello everyone! I hope you enjoyed Alexander's first lecture. I'm Ava and in this second lecture, Lecture 2, we're going to focus on this question of sequence modeling - how we can build neural networks that can handle and learn from sequential data. So in Alexander's first lecture he introduced the essentials of neural networks starting with perceptrons building up to feed forward models and how you can actually train these models and start to think about deploying them forward. Now we're going to turn our attention to specific types of problems that involve sequential processing of data and we'll realize why these types of problems require a different way of implementing and building neural networks from what we've seen so far. And I think some of the components in this lecture traditionally can be a bit confusing or daunting at first but what I really really want to do is to build this understanding up from the foundations walking through step by step developing intuition all the 